<a href="https://colab.research.google.com/github/cemsakarya/whitepaper-parkinglot/blob/main/Dunder_Mifflin_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
import random
import re
random.seed(2704)
number_of_rows = 10_000
from datetime import datetime
from datetime import timedelta
import requests
from bs4 import BeautifulSoup
import uuid

## Parsing the Wikipedia page for Paper Products and Pennsylvania Counties

In [29]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [30]:
response = requests.get(
url='https://en.wikipedia.org/wiki/Category:Paper_products',
)
soup = BeautifulSoup(response.content, 'html.parser')

# Get all the links
allLinks = soup.find(id="bodyContent").find_all("a")
random.shuffle(allLinks)
linkToScrape = 0

list_of_products = []
for i in allLinks:
    if str(i)[:15] == """<a href="/wiki/""":
        try:
            s = find_between( str(i), """href="/wiki/""", """" title=""" )
            if "category" in s.lower():
                pass
            else:
                list_of_products += [s]
        except:
            continue

In [31]:
print(f"Number of Products: {len(list_of_products)}")
print('\n10 Examples\n')
print(*list_of_products[:10], sep='\n')

Number of Products: 100

10 Examples

Chinet
Anaglypta
Holy_card
Receipt
Ticket_(admission)
Softwall
Duo-Tang
Coffee_cup_sleeve
Passbook
Container_compression_test


In [32]:
response = requests.get(
url='https://en.wikipedia.org/wiki/List_of_counties_in_Pennsylvania',
)
soup = BeautifulSoup(response.content, 'html.parser')

# Get all the links
allLinks = soup.find(id="bodyContent").find_all("a")
random.shuffle(allLinks)
linkToScrape = 0

list_of_counties = []
for i in allLinks:
    if str(i)[:15] == """<a href="/wiki/""":
        try:
            s = find_between( str(i), """href="/wiki/""", """" title=""" )
            if "category" in s.lower():
                pass
            elif not "_pennsylvania" in s.lower():
                pass
            else:
                list_of_counties += [s]
        except:
            continue

## Datetime

In [33]:
import random
import time
    
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y %I:%M %p', prop)

In [34]:
list_of_salesman = ["Dwight Schrute", "Jim Halpert", "Stanley Hudson", "Phyllis Vance", "Andrew Bernard"]

In [35]:
list_of_accountants = ["Kevin Malone", "Angela Martin", "Oscar Martinez"]

In [36]:
prices_of_products = dict(zip(list_of_products, np.random.randint(10,200,size=(number_of_rows, 1)[0])))

## Customers

In [37]:
word_site = "https://www.mit.edu/~ecprice/wordlist.100000"

response = requests.get(word_site)
WORDS = response.content.splitlines()

def business_name_generator():
    word = random.choice(WORDS)
    WORDS.remove(word)
    return word.decode("utf-8")  + ".inc"

In [38]:
!pip install names
import names

def name_generator():
    return names.get_full_name()

In [39]:
list_of_job_titles = ["General Manager","Administrative Assistant","Executive Assistant","Marketing Manager",
              "Customer Service Representative","Nurse Practitioner",
              "Sales Manager","Data Entry Clerk","Office Assistant", "Supply Manager", "Supply Manager", np.nan]

In [40]:
class _customer:
    def __init__(self):
        self.name = business_name_generator()
        self.customer_id = uuid.uuid4().hex
        self.county = random.choice(list_of_counties)
        self.Main_Customer_Representative = name_generator()
        self.Secondary_Customer_Representative = name_generator()
        self.Tertiary_Customer_Representative = name_generator()
        self.Main_Customer_Representative_Job_Title  = random.choice(list_of_job_titles)
        self.Secondary_Customer_Representative_Job_Title  = random.choice(list_of_job_titles)
        self.Tertiary_Customer_Representative_Job_Title   = random.choice(list_of_job_titles)
        self.salesman = random.choice(list_of_salesman)

In [41]:
i = 0
customer_df = pd.DataFrame([])
while i < 500:
    customer = _customer()
    temp = pd.DataFrame([[customer.name, customer.customer_id, customer.county, customer.Main_Customer_Representative,
                    customer.Secondary_Customer_Representative, customer.Tertiary_Customer_Representative,
                    customer.Main_Customer_Representative_Job_Title, customer.Secondary_Customer_Representative_Job_Title,
                    customer.Tertiary_Customer_Representative_Job_Title, customer.salesman]])
    
    customer_df = customer_df.append(temp) 
    i += 1

In [42]:
customer_df.columns = ["name", "customer_id", "county", "Main_Customer_Representative",
                    "Secondary_Customer_Representative", "Tertiary_Customer_Representative",
                    "Main_Customer_Representative_Job_Title", "Secondary_Customer_Representative_Job_Title",
                    "Tertiary_Customer_Representative_Job_Title", "Salesman"]

## Sales

In [43]:
class _sales:
    def __init__(self, salesman, customer_representative, customer_id):
        self.sales_id = uuid.uuid4().hex
        self.sales_datetime_str = random_date("1/1/2008 12:00 PM", "1/1/2010 12:00 PM", random.random())
        
        self.sales_datetime = datetime.strptime(self.sales_datetime_str, '%m/%d/%Y %I:%M %p')
        self.product = random.choice(list_of_products)
        self.number_of_units = np.random.randint(100,2000,size=(1, 1))[0][0]
        self.price_by_unit = prices_of_products[self.product]
        self.revenue = self.number_of_units * self.price_by_unit

        self.salesman = salesman
        
        self.delivery_how_many_days_later = timedelta(days = int(np.random.randint(1,30,size=(1, 1))[0][0])  )
        self.delivery_date = ((self.sales_datetime + self.delivery_how_many_days_later).date()).strftime("%m/%d/%Y")
        
        self.data_entry_how_many_days_later = timedelta(days = int(np.random.randint(1,10,size=(1, 1))[0][0])  )
        self.data_entry_date = ((self.sales_datetime + self.data_entry_how_many_days_later).date()).strftime("%m/%d/%Y")
        self.data_entry_officer = random.choice(list_of_accountants)
        
        self.customer_representative = customer_representative
        self.customer_id = customer_id

In [44]:
i = 0
sales_df = pd.DataFrame([])
while i < number_of_rows:
    
    salesman = random.choice(list_of_salesman)
    customer_id = random.choice(customer_df[customer_df["Salesman"] == salesman]["customer_id"].tolist())
    customer_representative = random.choice(customer_df[customer_df["customer_id"] == customer_id]["Main_Customer_Representative"].tolist() + customer_df[customer_df["customer_id"] == customer_id]["Secondary_Customer_Representative"].tolist() + customer_df[customer_df["customer_id"] == customer_id]["Tertiary_Customer_Representative"].tolist())
    
    sale = _sales(salesman, customer_representative, customer_id)
    temp = pd.DataFrame([[ sale.sales_id,  sale.sales_datetime_str, sale.sales_datetime, sale.product, 
                          sale.number_of_units, sale.price_by_unit, sale.revenue,
                            sale.salesman, sale.delivery_how_many_days_later,
                            sale.delivery_date, sale.data_entry_how_many_days_later, sale.data_entry_date, 
                          sale.data_entry_officer, sale.customer_representative, sale.customer_id]])
    
    sales_df = sales_df.append(temp) 
    i += 1

In [45]:
sales_df.columns = ["Order ID","Order Entry Datetime Str","Order Entry Datetime","Product","# of Units", "Revenue",
                    "Price by Unit","Salesman",
                          "Delivery How Many Dates Later", "Delivery Date", 
                    "Data Entry How Many Dates Later",
                    "Data Entry Date", "Data Entry Officer", "Customer Representative", "Customer ID"]

## Warehouse

In [46]:
class _warehouse:
    def __init__(self, order_id, order_date, latency_customer):
        self.order_id = order_id
        self.order_date = datetime.strptime(order_date, '%m/%d/%Y %I:%M %p')
        self.latency_customer = latency_customer
        self.quality_control = None   
        
        self.latency_load = timedelta(days = int(np.random.randint(0,5,size=(1, 1))[0][0]))      
        
        self.load_date = ((self.order_date + self.latency_customer + self.latency_load).date()).strftime("%m/%d/%Y")       
        
        self.latency_delivery = timedelta(days = int(np.random.randint(0,5,size=(1, 1))[0][0])) 
        
        
        self.delivery_date = ((self.order_date + self.latency_customer + self.latency_load + self.latency_delivery).date()).strftime("%m/%d/%Y")

In [47]:
i = 0
warehouse_logs = pd.DataFrame([])
for index, row in sales_df.iterrows():
    log = _warehouse(row["Order ID"], row["Order Entry Datetime Str"], row["Delivery How Many Dates Later"])
    
    temp = pd.DataFrame([[ log.order_id, log.order_date, log.latency_customer, log.quality_control,
                         log.latency_load, log.load_date, log.latency_delivery, log.delivery_date]])
    
    warehouse_logs = warehouse_logs.append(temp)

In [48]:
warehouse_logs.columns = ["Order ID", 
                          "Order Date",
                          "Latency Customer",
                          "Quality Control Check is Done",
                          "Latency Load",
                          "Load Date",
                          "Latency Delivery",
                          "Delivery Date"]

In [49]:
warehouse_logs.columns

Index(['Order ID', 'Order Date', 'Latency Customer',
       'Quality Control Check is Done', 'Latency Load', 'Load Date',
       'Latency Delivery', 'Delivery Date'],
      dtype='object')

##Payments - Accounts Receivable

In [115]:
class account:
  def __init__(self, customer_id, datetime):
    self.customer_id  = customer_id
    self.sales_amount = sales_df[(sales_df["Customer ID"] == customer_id) & (sales_df["Order Entry Datetime"] < datetime)]["Revenue"].sum()
    self.payments_amount =  payments_df[(payments_df["Customer ID"] == customer_id) & (payments_df["Payments Datetime"] < datetime)]["Payment Amount"].sum()
    self.amount_due = self.sales_amount - self.payments_amount

In [102]:
class payment:
  def __init__(self, customer_id, datetime, amount_due):
    self.customer_id = customer_id
    self.payments_datetime = datetime
    self.amount_due = amount_due
  
  def amount(self):
    try:
      if self.amount_due < 1000:
        return self.amount_due
      return np.random.randint(self.amount_due/5,self.amount_due,size=(1, 1))[0][0]
    except:
      pass

In [118]:
payments_df = pd.DataFrame([], columns = ["Customer ID", "Payments Datetime", "Payment Amount"])
for i in pd.date_range("2008-01-02","2008-01-20",freq='d'):
  temp = sales_df[sales_df["Order Entry Datetime"] < i]
  for j in range(np.random.randint(5,100,size=(1, 1))[0][0]):
    customer_id = random.choice(list(set(temp["Customer ID"])))
    customer_account = account(customer_id, i)
    payment_instance = payment(customer_id, i, customer_account.amount_due)
    payment_amount = payment_instance.amount()
    if payment_amount > 0:
      payments_df = payments_df.append(pd.DataFrame([[customer_id, i, payment_amount]], columns = ["Customer ID", "Payments Datetime", "Payment Amount"]), ignore_index=True)
  

In [ ]:
class Location:

    def __init__(self, name, height, since):
        self.name = name
        self.height = height
        self.since = since

Akron = Location("Akron","branch",2006)
Scranton = Location("Scranton","branch",2006)
Camden = Location("Camden","former_branch"2005)
Buffalo = Location("Buffalo","former_branch",2000)
Rochester = Location("Rochester","branch",2004)
Utica = Location("Utica","branch",2002)
Albany = Location("Albany","branch",2003)
Syracuse = Location("Syracuse","branch",2007)
Binghamton = Location("Binghamton","former_branch",2000)
Pittsfield = Location("Pittsfield","former_branch",2000)
Nashua = Location("Nashua","branch",2010)
Stamford = Location("Stamford","former_branch",2011)
Yonkers = Location("Yonkers","former_branch",2009)
New_york = Location("New_york","Headquarters",1997)

**Store it all in a DB**


In [121]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [126]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"/content/drive/My Drive/Colab Notebooks/dundermifflin.db")


2.6.0


In [127]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [128]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [129]:
def main():
    database = r"/content/drive/My Drive/Colab Notebooks/dundermifflin.db"

    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        begin_date text,
                                        end_date text
                                    ); """

    sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    priority integer,
                                    status_id integer NOT NULL,
                                    project_id integer NOT NULL,
                                    begin_date text NOT NULL,
                                    end_date text NOT NULL,
                                    FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")
